In [1]:
import os
import gym
import numpy as np
import tensorflow as tf
from rl.replay_buffer import ReplayBuffer

In [2]:
# Set random seeds
def set_random_seed(seed):
    tf.set_random_seed(seed)
    np.random.seed(seed)  
set_random_seed(0)

In [3]:
# Configuragtions
LEARNING_RATE = 0.01

In [4]:
# Setup
env = gym.make("BipedalWalker-v2")
replay_buffer = ReplayBuffer(
    observation_shape=env.observation_space.shape,
    action_shape=env.action_space.shape
)



In [5]:
# Create actor network
with tf.variable_scope("actor"):
    inputs = tf.keras.layers.Input(shape=(env.observation_space.shape))
    inputs = tf.keras.layers.Flatten()
    

In [ ]:
observation = env.reset()
done = False
while not done:
    action = env.action_space.sample()
    observation, reward, done, _ = env.step(action)
    env.render()
env.close()